In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
import os
import sys
import time
import mahotas as mh
from skimage import measure
from tqdm.notebook import tqdm
from image.controller import ImageController

/home/kunal.jain001/miniconda3/envs/O/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [4]:
data_dir = '/raid/mpsych/RISTERLAB/fly_segmentation_experiments/data_dir/nina_d1_raw/'
save_dir = '/raid/mpsych/RISTERLAB/fly_segmentation_experiments/data_dir/04-14/nina_d1_mutant/binary_mask_4/'

In [5]:
LOGGING_IN_NOTEBOOK = True
log_path = save_dir + 'log.txt'
crop_path = save_dir + 'original_crop/'
binary_mask_path = save_dir + 'binary_mask/'
compare_plot_path = save_dir + 'compare_plots/'
compare_labeled_plot_path = save_dir + 'compare_labeled_plots/'

In [6]:
if not LOGGING_IN_NOTEBOOK:
    sys.stdout = open(log_path, 'w')

In [7]:
# NOTE: one single ommatidium has 7 rhabdomeres.
# Value NUM_RHABDOMERE and values related to threshold are not to be changed
NUM_RHABDOMERE = 7

THRESHOLD_BRACKET = 40
LEAST_POSSIBLE_THRESHOLD = 1
MOST_POSSIBLE_THRESHOLD = 255

# Change these values as per different datasets
# Values for Wild_Type_CS_1
#     REMOVE_SMALL_REGION = True
#     REGION_SIZE = 750
#     LEAST_PIXEL_COUNT = 20000
#     LEAST_OMMATIDIA_COUNT = 1
# Values for NINA_D1_MUTANT
#     CROP_SIZE = (512, 512)
#     REMOVE_SMALL_REGION = False
#     REGION_SIZE = 750
#     LEAST_PIXEL_COUNT = 20000
#     LEAST_OMMATIDIA_COUNT = 12
CROP_SIZE = (512, 512)
REMOVE_SMALL_REGION = False
REGION_SIZE = 750
LEAST_PIXEL_COUNT = 20000
LEAST_OMMATIDIA_COUNT = 12

In [8]:
is_mask_bad = True
good_mask_count, bad_mask_count = 0, 0
file_count = 0

for file in tqdm(sorted(os.listdir(data_dir))):
    start_time = time.time()
    
    if file.endswith(".tif"):
        
        num_rhabdomeres, num_ommatidia = 0, 0
        best_threshold, best_accuracy, threshold_value = 0, 0, 0
        
        file_count += 1
        print("Image Name: " + file)
        
        # Read file
        img_file = os.path.join(data_dir, file)
        data = ImageController.read(img_file)
        
        # Normalize and crop
        # crop size defaults to (512, 512)
        normalized_data = ImageController.normalize(data)
        crop_data = ImageController.center_crop(normalized_data, crop_size=CROP_SIZE)
        
        # Save original image cropped
        data_crop = ImageController.center_crop(data, crop_size=CROP_SIZE)
        ImageController.save(os.path.join(crop_path, file), data_crop)
        
        # Apply guassian filter
        # can pass sigma as argument, defaults to 3
        smoothed_data = ImageController.smooth(crop_data)
        
        # Playing around with threshold values.
        # Start with mean threshold
        # Checking accuracy for 40 threshold up and down the mean value
        # Has to be done due to contrast variation in data
        threshold_value = int(smoothed_data.mean())
        threshold_start = LEAST_POSSIBLE_THRESHOLD
        threshold_limit = MOST_POSSIBLE_THRESHOLD
        if (threshold_value - THRESHOLD_BRACKET > LEAST_POSSIBLE_THRESHOLD):
            threshold_start = int(threshold_value) - THRESHOLD_BRACKET
        if (threshold_value + THRESHOLD_BRACKET < MOST_POSSIBLE_THRESHOLD):
            threshold_limit = int(threshold_value) + THRESHOLD_BRACKET
        
        # Loop over all threshold values and get threshold with best accuracy
        for threshold in range(threshold_start, threshold_limit, 1):
            # Apply thresholding
            threshold_data = ImageController.threshold(smoothed_data, threshold_value=threshold)
            # region labelling
            labeled_data, nr_count = ImageController.label(threshold_data)
            # remove regions
            # whether to remove small or large regions should be decided on the basis of dataset
            if REMOVE_SMALL_REGION:
                selected_labeled_data = ImageController.remove_small_regions(labeled_data, region_size=REGION_SIZE)
            else :
                selected_labeled_data = ImageController.remove_large_regions(labeled_data, region_size=REGION_SIZE)
            # Get binary mask of region removed labeled data
            binary_mask_data = ImageController.binary_mask(selected_labeled_data)
            # Apply close operation on binary mask
            segmented_image = ImageController.close_binary_mask(binary_mask_data)
            # count pixels in mask
            num_pixels = int(np.sum(segmented_image))
            # check accuracy of mask
            test_accuracy, test_num_rhabdomeres = ImageController.check_accuracy(binary_mask_data)
            # get best accuracy and satisfy least pixel count value
            if test_accuracy > best_accuracy and num_pixels > LEAST_PIXEL_COUNT:
                best_accuracy = test_accuracy
                best_threshold = threshold
                num_rhabdomeres = test_num_rhabdomeres
           
        # Run pipeline one last time for best threshold value
        threshold_data = ImageController.threshold(smoothed_data, threshold_value=best_threshold)
        labeled_data, nr_count = ImageController.label(threshold_data)
        if REMOVE_SMALL_REGION:
            selected_labeled_data = ImageController.remove_small_regions(labeled_data, region_size=REGION_SIZE)
        else :
            selected_labeled_data = ImageController.remove_large_regions(labeled_data, region_size=REGION_SIZE)
        binary_mask_data = ImageController.binary_mask(selected_labeled_data)
        closed_binary_mask_data = ImageController.close_binary_mask(binary_mask_data)
        segmented_image = ImageController.binary_image(closed_binary_mask_data)
        
        # count number of pixels in segmentation mask
        num_pixels = np.sum(segmented_image)
        
        # Save binary_mask, compare_labeled_plot and conpare_plot for best accuracy threshold
        compare_labeled_path = os.path.join(compare_labeled_plot_path, file)
        compare_labeled_plot_name = compare_labeled_path.split('.')[0] + '.png'
        
        compare_path = os.path.join(compare_plot_path, file)
        compare_plot_name = compare_path.split('.')[0] + '.png'
        
        binary_mask_name = os.path.join(binary_mask_path, file)
        
        # Save compare_labeled_plots for all images
        ImageController.display_and_save_compare_labeled(data_crop, 
                                                         binary_mask_data, 
                                                         segmented_image, 
                                                         compare_labeled_plot_name,
                                                         show_in_notebook=False)
        
        print("Threshold mean: ", smoothed_data.mean())
        print("Threshold best: ", best_threshold)
        print("Accuracy of mask: ", best_accuracy)
        
        num_ommatidia = round(num_rhabdomeres / NUM_RHABDOMERE)
        # if number of ommatidia in image and number of pixels in image
        # satisfy least count, then segmentation mask is good.
        if num_pixels > LEAST_PIXEL_COUNT and num_ommatidia >= LEAST_OMMATIDIA_COUNT:
            good_mask_count += 1
            print("Good segmentation mask")
            # Save binary mask and compare plot only for good mask image
            ImageController.save_compare(data_crop, segmented_image, compare_plot_name)
            ImageController.save(binary_mask_name, segmented_image)
        else:
            bad_mask_count += 1
            print("Bad segmentation mask")
            
        print("Possible number of rhabdomeres: ", num_rhabdomeres)
        print("Total number of pixels in the mask: ", num_pixels)
        
        end_time = time.time()
        print("Total Time Taken: ", (end_time - start_time), " seconds")
        
        print("Number of good masks so far: ", good_mask_count)
        print("Number of bad masks so far: ", bad_mask_count)
        print("Total files processed so far: ", file_count)
        
        print("---------------------------------------------------------")

print("Number of good masks: ", good_mask_count)
print("Number of bad masks: ", bad_mask_count)
print("Total processed images:", file_count)

  0%|          | 0/301 [00:00<?, ?it/s]

Image Name: 000000_000000_000903_000000.tif
Threshold mean:  29.143386126437907
Threshold best:  30
Accuracy of mask:  0.9263157894736842
Good segmentation mask
Possible number of rhabdomeres:  88
Total number of pixels in the mask:  20322
Total Time Taken:  3.4754114151000977  seconds
Number of good masks so far:  1
Number of bad masks so far:  0
Total files processed so far:  1
---------------------------------------------------------
Image Name: 000000_000000_000905_000000.tif
Threshold mean:  29.24035932816866
Threshold best:  33
Accuracy of mask:  0.8837209302325582
Good segmentation mask
Possible number of rhabdomeres:  114
Total number of pixels in the mask:  27046
Total Time Taken:  3.5368261337280273  seconds
Number of good masks so far:  2
Number of bad masks so far:  0
Total files processed so far:  2
---------------------------------------------------------
Image Name: 000000_000000_000906_000000.tif
Threshold mean:  29.497295056507273
Threshold best:  33
Accuracy of mask: 

Threshold mean:  19.89148121875721
Threshold best:  19
Accuracy of mask:  0.8953488372093024
Bad segmentation mask
Possible number of rhabdomeres:  77
Total number of pixels in the mask:  21761
Total Time Taken:  2.714087724685669  seconds
Number of good masks so far:  17
Number of bad masks so far:  3
Total files processed so far:  20
---------------------------------------------------------
Image Name: 000000_000000_001003_000000.tif
Threshold mean:  16.875069874983566
Threshold best:  27
Accuracy of mask:  0.9421965317919075
Good segmentation mask
Possible number of rhabdomeres:  163
Total number of pixels in the mask:  21899
Total Time Taken:  3.0454351902008057  seconds
Number of good masks so far:  18
Number of bad masks so far:  3
Total files processed so far:  21
---------------------------------------------------------
Image Name: 000000_000000_001005_000000.tif
Threshold mean:  18.203727273277316
Threshold best:  19
Accuracy of mask:  0.9313725490196079
Good segmentation mask

Threshold mean:  14.267521773502507
Threshold best:  24
Accuracy of mask:  0.9202453987730062
Good segmentation mask
Possible number of rhabdomeres:  150
Total number of pixels in the mask:  21666
Total Time Taken:  2.919705390930176  seconds
Number of good masks so far:  33
Number of bad masks so far:  6
Total files processed so far:  39
---------------------------------------------------------
Image Name: 000000_000000_001082_000000.tif
Threshold mean:  13.022487415062203
Threshold best:  12
Accuracy of mask:  0.9166666666666666
Good segmentation mask
Possible number of rhabdomeres:  88
Total number of pixels in the mask:  21690
Total Time Taken:  2.7412476539611816  seconds
Number of good masks so far:  34
Number of bad masks so far:  6
Total files processed so far:  40
---------------------------------------------------------
Image Name: 000000_000000_001089_000000.tif
Threshold mean:  12.80494096514943
Threshold best:  13
Accuracy of mask:  0.9099099099099099
Good segmentation mas

Threshold mean:  72.32770155637691
Threshold best:  76
Accuracy of mask:  0.9202898550724637
Good segmentation mask
Possible number of rhabdomeres:  127
Total number of pixels in the mask:  30020
Total Time Taken:  3.9287350177764893  seconds
Number of good masks so far:  41
Number of bad masks so far:  17
Total files processed so far:  58
---------------------------------------------------------
Image Name: 000000_000000_001157_000000.tif
Threshold mean:  71.21069531388335
Threshold best:  71
Accuracy of mask:  0.9452054794520548
Bad segmentation mask
Possible number of rhabdomeres:  69
Total number of pixels in the mask:  23148
Total Time Taken:  3.4810519218444824  seconds
Number of good masks so far:  41
Number of bad masks so far:  18
Total files processed so far:  59
---------------------------------------------------------
Image Name: 000000_000000_001160_000000.tif
Threshold mean:  71.37760293614733
Threshold best:  72
Accuracy of mask:  0.9390243902439024
Bad segmentation mask

...

Threshold mean:  32.65107714125525
Threshold best:  46
Accuracy of mask:  0.9495798319327731
Good segmentation mask
Possible number of rhabdomeres:  226
Total number of pixels in the mask:  20233
Total Time Taken:  3.961871385574341  seconds
Number of good masks so far:  144
Number of bad masks so far:  66
Total files processed so far:  210
---------------------------------------------------------
Image Name: 000000_000000_001789_000000.tif
Threshold mean:  37.53415312522497
Threshold best:  51
Accuracy of mask:  0.9471544715447154
Good segmentation mask
Possible number of rhabdomeres:  233
Total number of pixels in the mask:  21740
Total Time Taken:  4.139067649841309  seconds
Number of good masks so far:  145
Number of bad masks so far:  66
Total files processed so far:  211
---------------------------------------------------------
Image Name: 000000_000000_001791_000000.tif
Threshold mean:  38.75508038202923
Threshold best:  52
Accuracy of mask:  0.9297520661157025
Good segmentation

Threshold mean:  61.73274895210407
Threshold best:  78
Accuracy of mask:  0.9282700421940928
Good segmentation mask
Possible number of rhabdomeres:  220
Total number of pixels in the mask:  20588
Total Time Taken:  4.388967514038086  seconds
Number of good masks so far:  162
Number of bad masks so far:  67
Total files processed so far:  229
---------------------------------------------------------
Image Name: 000000_000000_001877_000000.tif
Threshold mean:  59.762888478708796
Threshold best:  75
Accuracy of mask:  0.9425287356321839
Good segmentation mask
Possible number of rhabdomeres:  246
Total number of pixels in the mask:  22383
Total Time Taken:  4.377039909362793  seconds
Number of good masks so far:  163
Number of bad masks so far:  67
Total files processed so far:  230
---------------------------------------------------------
Image Name: 000000_000000_001879_000000.tif
Threshold mean:  52.22024133004549
Threshold best:  67
Accuracy of mask:  0.9380165289256198
Good segmentatio

Threshold mean:  46.002433737555705
Threshold best:  47
Accuracy of mask:  0.9337748344370861
Good segmentation mask
Possible number of rhabdomeres:  141
Total number of pixels in the mask:  23234
Total Time Taken:  4.20912766456604  seconds
Number of good masks so far:  181
Number of bad masks so far:  67
Total files processed so far:  248
---------------------------------------------------------
Image Name: 000000_000000_001934_000000.tif
Threshold mean:  50.106615872610185
Threshold best:  64
Accuracy of mask:  0.9274809160305344
Good segmentation mask
Possible number of rhabdomeres:  243
Total number of pixels in the mask:  24164
Total Time Taken:  4.3757688999176025  seconds
Number of good masks so far:  182
Number of bad masks so far:  67
Total files processed so far:  249
---------------------------------------------------------
Image Name: 000000_000000_001938_000000.tif
Threshold mean:  64.21310518600129
Threshold best:  78
Accuracy of mask:  0.9307958477508651
Good segmentati

Threshold mean:  54.10690327179737
Threshold best:  66
Accuracy of mask:  0.9301470588235294
Good segmentation mask
Possible number of rhabdomeres:  253
Total number of pixels in the mask:  22838
Total Time Taken:  4.348074674606323  seconds
Number of good masks so far:  200
Number of bad masks so far:  67
Total files processed so far:  267
---------------------------------------------------------
Image Name: 000000_000000_002009_000000.tif
Threshold mean:  64.88302632391235
Threshold best:  70
Accuracy of mask:  0.9273743016759777
Good segmentation mask
Possible number of rhabdomeres:  166
Total number of pixels in the mask:  20657
Total Time Taken:  4.44982647895813  seconds
Number of good masks so far:  201
Number of bad masks so far:  67
Total files processed so far:  268
---------------------------------------------------------
Image Name: 000000_000000_002010_000000.tif
Threshold mean:  62.03217189041251
Threshold best:  76
Accuracy of mask:  0.921146953405018
Good segmentation m

Threshold mean:  40.41131784802392
Threshold best:  46
Accuracy of mask:  0.9011406844106464
Good segmentation mask
Possible number of rhabdomeres:  237
Total number of pixels in the mask:  26907
Total Time Taken:  4.1201722621917725  seconds
Number of good masks so far:  218
Number of bad masks so far:  68
Total files processed so far:  286
---------------------------------------------------------
Image Name: 000000_000000_002101_000000.tif
Threshold mean:  43.06394457031083
Threshold best:  51
Accuracy of mask:  0.8596491228070176
Good segmentation mask
Possible number of rhabdomeres:  245
Total number of pixels in the mask:  23786
Total Time Taken:  4.139944553375244  seconds
Number of good masks so far:  219
Number of bad masks so far:  68
Total files processed so far:  287
---------------------------------------------------------
Image Name: 000000_000000_002105_000000.tif
Threshold mean:  49.80774527155001
Threshold best:  60
Accuracy of mask:  0.9010238907849829
Good segmentatio